In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification 

In [2]:
import torch 
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd

In [3]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [4]:
tokens = tokenizer.encode('i love it, its awesome', return_tensors='pt')

In [5]:
tokenizer.decode(tokens[0])

'[CLS] i love it, its awesome [SEP]'

In [6]:
result = model(tokens)

In [7]:
result.logits

tensor([[-2.5852, -2.6762, -0.7653,  1.6446,  3.5830]],
       grad_fn=<AddmmBackward0>)

In [8]:
torch.argmax(result.logits)

tensor(4)

In [9]:
rating =int(torch.argmax(result.logits))+1

In [10]:
rating

5

In [11]:
print(f'rating is {rating} stars')

rating is 5 stars


In [12]:

import requests
from bs4 import BeautifulSoup
import re

url = 'https://www.yelp.com/biz/social-brew-cafe-pyrmont'  
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
regex = re.compile('.*comment.*')
review_elements = soup.find_all('p', {'class': regex})
reviews = [review.text for review in review_elements]

for review in reviews:
    print(review)



Very cute coffee shop and restaurant. They have a lovely outdoor seating area and several tables inside.  It was fairly busy on a Tuesday morning but we were to grab the last open table. The server was so enjoyable, she chatted and joked with us and provided fast service with our ordering, drinks and meals. The food was very good. We ordered a wide variety and every meal was good to delicious. The sweet potato fries on the Chicken Burger plate were absolutely delicious, some of the best I've ever had. I definitely enjoyed this cafe, the outdoor seating, the service and the food!!
Six of us met here for breakfast before our walk to Manly. We were enjoying visiting with each other so much that I apologize for not taking any photos. We all enjoyed our food, as well as our coffee and tea drinks.We were greeted immediately by a friendly server asking if we would like to sit inside or out. We said we would like inside, but weren't exactly sure how many were joining us yet- at least 4. We wer

In [13]:
import numpy as np
import pandas as pd

In [15]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [16]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [17]:
sentiment_score(df['review'].iloc[1])

4

In [18]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [19]:
df

,review,sentiment
0,Very cute coffee shop and restaurant. They hav...,4
1,Six of us met here for breakfast before our wa...,4
2,We came for brunch and they ran out of seven s...,2
3,Great place with delicious food and friendly s...,5
4,The food was delicious. The ricotta pancakes w...,4
5,Great food amazing coffee and tea. Short walk ...,5
6,Ricotta hot cakes! These were so yummy. I ate ...,5
7,"Great service, lovely location, and really ama...",5
8,We came for brunch twice in our week-long visi...,4
9,It was ok. Had coffee with my friends. I'm new...,3


In [20]:
df['review'].iloc[3]

'Great place with delicious food and friendly staff. It is small but has outdoor seating and a relaxed ambiance. Perfect place to enjoy a cup of coffee. I am visiting Sydney for the first time but this place seems like is a local favorite.'